In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [3]:
anss = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=99,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_loc=False,
    include_depth = True
)

Generating the catalog...
Downloading...
train


100%|██████████| 59/59 [00:00<00:00, 280.19it/s]


val


100%|██████████| 19/19 [00:00<00:00, 363.40it/s]


test


100%|██████████| 19/19 [00:00<00:00, 383.31it/s]


Success!
Catalog saved to /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog


In [7]:
anss.train[0].extra_feat

tensor([ 56900.,  97700., 174400.,  36300.,  33000.,  33000.,  10000.,  33000.,
         33000.,    300., 194000., 169100.,  29500.,  10000.,  33000., 162500.,
         22000.,  33000.,  10600., 160900.,  36600.,  57100.,  49600.,  44000.,
         42400.,  41300.,   2400.,  47200.,  41700.,  40500.,  11700.,  33000.,
         77100.,   9200.,  75400., 166500.,  20700.,   2600.,  31400.,  73400.,
         10000.,  17500., 167900., 111400.,   9200.,  23600.,  19300.,  39100.,
         33000.,  45000.,  10000., 175000., 167400.,  29900.,  38400.,  50800.,
        161500.,  94500.,   7400., 142900.,  41600.,  62900.,      0.,  62300.,
         30300.,  56300., 175800.,  52500., 171600., 172800., 168800.,  10000.,
        105600.,  37300.,  59000.,  75000.,  59700.,  33000.,  33000.,  59300.,
         50000.,  51300.,  23000.,  33000.,   5000.,  16000.,  16000.,   5000.,
         33000.,  33000.,  10100.,  33000.,  46300.,  53400.,  33000.,  46600.,
         39900.,  10000.,      0.,  3300

In [9]:

dl_train = anss.train.get_dataloader( batch_size=200, shuffle=False)
dl_val = anss.val.get_dataloader(batch_size=200, shuffle=False)
dl_test = anss.test.get_dataloader(batch_size=200, shuffle=False)

In [10]:
T = anss.train[0].t_end
N = np.mean([len(seq) for seq in anss.train])
mag_mean = np.mean([anss.train[0].mag.mean().item() for seq in anss.train])
tau_mean = T/N

model = eq.models.RecurrentTPP(
    num_extra_features = 1,
    mag_mean = mag_mean,
    tau_mean = tau_mean,
    mag_completeness=anss.metadata['mag_completeness'],
    learning_rate=1e-3,
)

In [12]:
   # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=1000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(model, dl_train, dl_val)
checkpoint.best_model_path
model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(model, dl_test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.4 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.6 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.6 K                                                                                                
Total estimated model params size (MB): 0

Output()

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/
data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be
a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

RuntimeError: Tensors must have same number of dimensions: got 3 and 2

AttributeError: 'DataLoader' object has no attribute 'inter_times'

In [ ]:
fig, ax = plt.subplots()
ax.plot(running_training_loss, label="Training loss")
ax.plot(running_validation_loss, label="Validation loss")
ax.legend()
ax.set(
    xlabel="Epoch",
    ylabel="NLL per unit time", 
);